Package imports

In [26]:
import gc
import os
from datetime import datetime, timedelta
import netCDF4 as nc
import numpy as np
import pandas as pd
from geopy import Point
from geopy.distance import great_circle
from scipy.spatial import cKDTree
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet, Ridge, Lasso, BayesianRidge, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBRegressor
import lightgbm as lgb
import time
import math

Function to pre-process spatial data

In [2]:
# Precompute the KDTree and valid_time differences
def precompute_kdtree_and_time_diffs(uwnd_nc_file_path):
    try:
        print("Precomputing KDTree and time differences...")
        # Load the NetCDF file
        ds = nc.Dataset(uwnd_nc_file_path)

        # Extract the valid_time, latitudes, and longitudes from the NetCDF file
        valid_time = ds.variables['valid_time'][:]  # Assuming 'valid_time' is the variable name for time
        latitudes = ds.variables['latitude'][:]
        longitudes = ds.variables['longitude'][:]

        # Convert valid_time from seconds since 1970-01-01 to datetime
        base_time = datetime(1970, 1, 1)
        valid_time_dt = np.array([base_time + timedelta(seconds=int(ts)) for ts in valid_time], dtype='datetime64[ns]')

        # Create a KDTree for fast spatial lookup
        lat_lon_pairs = np.array([(lat, lon) for lat in latitudes for lon in longitudes])
        tree = cKDTree(lat_lon_pairs)

        print("KDTree and time differences precomputed successfully.")
        return tree, valid_time_dt, latitudes, longitudes, lat_lon_pairs
    except Exception as e:
        print(f"Error precomputing KDTree and time differences: {e}")
        raise

uwnd_nc_file_path = '../data/raw/reanalyses/ERA5/era5_uwnd_2023.nc'
vwnd_nc_file_path = '../data/raw/reanalyses/ERA5/era5_vwnd_2023.nc'
try:
    tree, valid_time_dt, latitudes, longitudes, lat_lon_pairs = precompute_kdtree_and_time_diffs(uwnd_nc_file_path)
except Exception as e:
    print(f"Error precomputing KDTree and time differences: {e}")
    raise


Precomputing KDTree and time differences...
KDTree and time differences precomputed successfully.


Function to extract wind components at a given lat/lon (preloads reanalysis netCDFs also)

In [3]:
uwnd_ds = nc.Dataset(uwnd_nc_file_path)
vwnd_ds = nc.Dataset(vwnd_nc_file_path)

uwnd_array = uwnd_ds.variables['u'][:, 0, :, :]  # Assuming 'u' is the variable name for u-component wind and removing the pressure dimension
vwnd_array = vwnd_ds.variables['v'][:, 0, :, :]  # Assuming 'v' is the variable name for v-component wind and removing the pressure dimension

# Function to extract wind components
def extract_wind_components(lat, lon, dt, tree, valid_time_dt, latitudes, longitudes, lat_lon_pairs):
    try:
        # Convert the given datetime to a numpy datetime64 object
        row_datetime = np.datetime64(dt)

        # Find the value in the valid_time dimension closest in time to the datetime in the dataframe
        time_diffs = np.abs(valid_time_dt - row_datetime)
        closest_time_index = np.argmin(time_diffs)

        # Check if the calculated index is within the bounds of the uwnd_array
        if closest_time_index < 0 or closest_time_index >= uwnd_array.shape[0]:
            raise ValueError("The given datetime is out of bounds for the NetCDF data")

        # Select the corresponding netCDF slices
        uwnd_slice = uwnd_array[closest_time_index, :, :]
        vwnd_slice = vwnd_array[closest_time_index, :, :]

        # Find the grid cell of the netCDF slice closest to the given Latitude and Longitude position
        lat_lon = (lat, lon)
        _, closest_point_index = tree.query(lat_lon)
        closest_lat, closest_lon = lat_lon_pairs[closest_point_index]

        # Find the index of the closest latitude/longitude pair in the arrays
        lat_index = np.where(latitudes == closest_lat)[0][0]
        lon_index = np.where(longitudes == closest_lon)[0][0]

        # Extract the u and v wind components
        u_wind = uwnd_slice[lat_index, lon_index]
        v_wind = vwnd_slice[lat_index, lon_index]

        # Round wind components to 4 decimal places
        u_wind = round(u_wind, 4)
        v_wind = round(v_wind, 4)

        return u_wind, v_wind
    except Exception as e:
        print(f"Error extracting wind components: {e}")
        raise

Function to calculate new position from current position, displacement, and heading

In [4]:
def calculate_new_position(current_position, displacement, heading):
    R = 6371000  # Earth's radius in meters
    
    lat1 = math.radians(current_position[0])
    lon1 = math.radians(current_position[1])
    heading_rad = math.radians(heading)
    
    lat2 = math.asin(math.sin(lat1) * math.cos(displacement / R) +
                     math.cos(lat1) * math.sin(displacement / R) * math.cos(heading_rad))
    
    lon2 = lon1 + math.atan2(math.sin(heading_rad) * math.sin(displacement / R) * math.cos(lat1),
                             math.cos(displacement / R) - math.sin(lat1) * math.sin(lat2))
    
    return math.degrees(lat2), math.degrees(lon2)

Iterative predictor function

In [17]:
def iterative_prediction(val_data, model, tree, valid_times, latitudes, longitudes, lat_lon_pairs, output_file_path, max_predicts=None):

    # Check if the output file exists and write the header only if it doesn't
    if not os.path.exists(output_file_path):
        with open(output_file_path, 'w') as file:
            file.write("Latitude,Longitude,Datetime,BuoyID,era5_uwnd,era5_vwnd,displacement,heading\n")

    # Print the input data length
    print(f"Number of rows in input validation data: {len(val_data)}")

    with open(output_file_path, 'a') as file:
        # Initialize starting conditions
        current_lat, current_lon = val_data.iloc[0][['Latitude', 'Longitude']]
        current_uwnd, current_vwnd = val_data.iloc[0][['era5_uwnd', 'era5_vwnd']]
        buoy_id = val_data.iloc[0]['BuoyID']
        previous_time = val_data.iloc[0]['datetime']
        
        print("\nInitial conditions:")
        print(f"Latitude: {current_lat:.2f}, Longitude: {current_lon:.2f}, Datetime: {previous_time}, BuoyID: {buoy_id}")

        # Initialize output count for tracking
        output_count = 0

        for i in range(1, len(val_data)):  # Start from the second row for time_step calculation
            # Check if the maximum number of predictions has been reached
            if max_predicts is not None and i >= max_predicts:
                print(f"Maximum number of predictions ({max_predicts}) reached. Stopping the script.")
                break

            next_row = val_data.iloc[i]
            
            # Calculate time_step between current and next row
            time_step = (next_row['datetime'] - previous_time).total_seconds()
            previous_time = next_row['datetime']  # Update previous_time for the next iteration
            
            # Convert wind components to magnitude and angle
            wind_magnitude = (current_uwnd**2 + current_vwnd**2) ** 0.5
            wind_angle = np.degrees(np.arctan2(current_vwnd, current_uwnd)) % 360  # Calculate wind angle in degrees
            
            # Prepare the input data for the model, including time_step
            input_data = pd.DataFrame({
                'wind_magnitude': [wind_magnitude],
                'wind_angle': [wind_angle],
                'time_step': [time_step]
            })
            
            # Predict the displacement and heading
            predicted_displacement, predicted_heading = model.predict(input_data)[0]
            predicted_lat, predicted_lon = calculate_new_position(
                (current_lat, current_lon),
                predicted_displacement,
                predicted_heading
            )
            
            # Extract wind components for the predicted location and next datetime
            predicted_wind_u, predicted_wind_v = extract_wind_components(
                predicted_lat, 
                predicted_lon, 
                next_row['datetime'],
                tree,
                valid_times,
                latitudes,
                longitudes,
                lat_lon_pairs
            )
            
            # Update current position and wind components for the next prediction
            current_lat, current_lon = predicted_lat, predicted_lon
            current_uwnd, current_vwnd = predicted_wind_u, predicted_wind_v

            # Write the prediction to the output file with all columns in val_data
            file.write(
                f"{current_lat},{current_lon},{next_row['datetime']},{buoy_id},{predicted_wind_u},{predicted_wind_v},{predicted_displacement},{predicted_heading}\n"
            )
            output_count += 1  # Increment output count for each successful prediction

        # Print the output data length after predictions
        print(f"Number of rows written to output file: {output_count}")

Splitting the data into training and validation sets

In [ ]:
# Load and prepare data
buoy_data = pd.read_csv('../processed_buoy_data.csv')

# Keep relevant columns for data preparation
columns_to_keep = ['Latitude', 'Longitude', 'BuoyID', 'datetime', 'wind_magnitude', 'wind_angle', 
                   'displacement', 'heading', 'era5_uwnd', 'era5_vwnd']
print("Dropping unused columns and retaining only:", columns_to_keep)
buoy_data = buoy_data[columns_to_keep]
buoy_data['datetime'] = pd.to_datetime(buoy_data['datetime'])
print("Datetime column successfully converted to datetime format.")

# Calculate time step between successive rows for each buoy
print("Calculating time step between successive observations.")
buoy_data = buoy_data.sort_values(by=['BuoyID', 'datetime']).reset_index(drop=True)
buoy_data['time_step'] = buoy_data.groupby('BuoyID')['datetime'].diff().dt.total_seconds()
buoy_data['time_step'] = buoy_data['time_step'].fillna(0)  # Fill NaN values with 0

# Split data by BuoyID into training and validation sets
print("Splitting data into training and validation sets by unique Buoy IDs.")
buoy_ids = buoy_data['BuoyID'].unique()
train_ids = np.random.choice(buoy_ids, size=int(len(buoy_ids) - 5), replace=False)
val_ids = np.setdiff1d(buoy_ids, train_ids)
print(f"Training IDs count: {len(train_ids)}, Validation IDs count: {len(val_ids)}")

# Separate train and validation data
train_data = buoy_data[buoy_data['BuoyID'].isin(train_ids)]
val_data = buoy_data[buoy_data['BuoyID'].isin(val_ids)]
print(f"Training data shape: {train_data.shape}, Validation data shape: {val_data.shape}")

# Define columns to keep for features and target
feature_columns = ['wind_magnitude', 'wind_angle', 'time_step']
target_columns = ['displacement', 'heading']

# Create training and validation feature and target sets
X_train = train_data[feature_columns]
y_train = train_data[target_columns]

X_val = val_data[feature_columns]
y_val = val_data[target_columns]

print(f"Prepared training data with feature columns {feature_columns}. Feature set shape: {X_train.shape}, Target set shape: {y_train.shape}")
print(f"Prepared validation data with feature columns {feature_columns}. Feature set shape: {X_val.shape}, Target set shape: {y_val.shape}")


Dropping unused columns and retaining only: ['Latitude', 'Longitude', 'BuoyID', 'datetime', 'wind_magnitude', 'wind_angle', 'displacement', 'heading', 'era5_uwnd', 'era5_vwnd']
Datetime column successfully converted to datetime format.
Calculating time step between successive observations.
Splitting data into training and validation sets by unique Buoy IDs.
Training IDs count: 281, Validation IDs count: 5
Training data shape: (1729023, 11), Validation data shape: (49677, 11)
Prepared training data without displacement and heading as features. Feature set shape: (1729023, 3), Target set shape: (1729023, 2)
Prepared validation data without displacement and heading as features. Feature set shape: (49677, 3), Target set shape: (49677, 2)


In [23]:
# Train the model
print("Training model...")
model = RandomForestRegressor()
model.fit(X_train, y_train)
print("Model training complete.")

Training model...
Model training complete.


Predicting

In [25]:
print("\nStarting iterative predictions on validation data...")

output_file_path = '../data/processed/predictions/'

# Create the output directory if it does not exist
if not os.path.exists(output_file_path):
    os.makedirs(output_file_path)

# Get unique buoyID values in the validation data
unique_buoy_ids = val_data['BuoyID'].unique()

# Loop through each buoyID and make predictions
for buoy_id in unique_buoy_ids:
    print(f"Processing BuoyID: {buoy_id}")
    
    # Filter validation data for the current buoyID
    val_data_buoy = val_data[val_data['BuoyID'] == buoy_id]
    
    # Create an output file path for the current buoyID
    output_file_path_buoy = f"{output_file_path}predicted_{buoy_id}.csv"
    
    # Print the output file path for the current buoyID
    print(f"Output file path for BuoyID {buoy_id}: {output_file_path_buoy}")

    # Run iterative prediction with error handling to track any potential issues
    try:
        iterative_prediction(
            val_data=val_data_buoy,
            model=model,
            tree=tree,
            valid_times=valid_time_dt,
            latitudes=latitudes,
            longitudes=longitudes,
            output_file_path=output_file_path_buoy,
            lat_lon_pairs=lat_lon_pairs,
            max_predicts=None
        )
        print(f"Predictions for BuoyID {buoy_id} completed and saved")
    except Exception as e:
        print(f"Error encountered for BuoyID {buoy_id}: {e}")


Starting iterative predictions on validation data...
Processing BuoyID: 902002
Output file path for BuoyID 902002: ../data/processed/predictions/predicted_902002.csv
Number of rows in input validation data: 17521

Initial conditions:
Latitude: 84.07, Longitude: -115.32, Datetime: 2023-01-01 00:00:46, BuoyID: 902002
Number of rows written to output file: 17520
Predictions for BuoyID 902002 completed and saved
Processing BuoyID: 300234066034140
Output file path for BuoyID 300234066034140: ../data/processed/predictions/predicted_300234066034140.csv
Number of rows in input validation data: 8829

Initial conditions:
Latitude: 79.49, Longitude: -7.59, Datetime: 2023-01-01 00:00:00, BuoyID: 300234066034140
Number of rows written to output file: 8828
Predictions for BuoyID 300234066034140 completed and saved
Processing BuoyID: 300234067525560
Output file path for BuoyID 300234067525560: ../data/processed/predictions/predicted_300234067525560.csv
Number of rows in input validation data: 229

I